## 0. Imports

In [8]:
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import numpy as np
import torch
import time
import json
from tqdm.notebook import tqdm

from dataset import LoadDataset
from MSGModel import Generator, Discriminator, DownSampleImage
from trainMSG import train_generator, train_discriminator, evaluate
from settings import DEVICE, MEAN, STD, BATCH_SIZE, LATENT_SIZE, EPOCHS, TRAINNING_DS_SIZE, RELOAD_DS_PER_EPOCH, LOSS_FN, OPTIMIZER, BETAS, D_LR, D_LR_DECAY, G_LR, G_LR_DECAY, CORES, LOAD_MODELS

# torch.set_num_threads(4)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Dataset

In [9]:
ds = LoadDataset('../data/train', MEAN, STD, length=TRAINNING_DS_SIZE)
loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=CORES)

val_ds = LoadDataset('../data/val', MEAN, STD)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=CORES)

print(f"Dataset size: {len(ds)}, loader size: {len(loader)}")
print(f"Validation dataset size: {len(val_ds)}, validation loader size: {len(val_loader)}")

Dataset size: 92032, loader size: 5752
Validation dataset size: 192, validation loader size: 12


## 2. Models

In [10]:
G, D, downsampler = Generator(), Discriminator(), DownSampleImage()
# print(G)
# print(D)
print(f"Num parameters in Discriminator: {sum(p.numel() for p in D.parameters())}")
print(f"Num parameters in Generator: {sum(p.numel() for p in G.parameters())}")

Num parameters in Discriminator: 23137153
Num parameters in Generator: 23053781


## 3. Training

In [11]:
d_losses = []
g_losses = []
r_scores = []
f_scores = []
d_optimizer = OPTIMIZER(D.parameters(), D_LR, betas=BETAS)
g_optimizer = OPTIMIZER(G.parameters(), G_LR, betas=BETAS)

try:
    if LOAD_MODELS:
        D.load_state_dict(torch.load('discriminator.pth'))
        G.load_state_dict(torch.load('generator.pth'))
        print("Model loaded")
    else:
        print("Trainning new model")
except:
    print("Trainning new model")
    pass

D.to(DEVICE)
G.to(DEVICE)

if not LOAD_MODELS:
    with open("log.json", "w") as fh:
        json.dump({"epochs": 0}, fh)
    start_epochs = 0
else:
    with open("log.json", "r") as fh:
        log = json.load(fh)
    start_epochs = log['epochs']
    for e in range(1, start_epochs+1):
        d_losses.append(log[str(e)]['d_loss'])
        g_losses.append(log[str(e)]['g_loss'])
        r_scores.append(log[str(e)]['r_score'])
        f_scores.append(log[str(e)]['f_score'])

    d_optimizer.param_groups[0]['lr'] -= D_LR_DECAY * start_epochs
    g_optimizer.param_groups[0]['lr'] -= G_LR_DECAY * start_epochs
    
    latent = torch.randn(BATCH_SIZE, *LATENT_SIZE).to(DEVICE)
    fake_images = G(latent)[0] * 0.5 + 0.5
    save_image(fake_images, 'current.png')

print(f"Start at epochs {start_epochs}")

Trainning new model
Start at epochs 0


In [ ]:
for epochs in range(start_epochs, EPOCHS):
    start = time.time()
    # Reload dataset
    if RELOAD_DS_PER_EPOCH:
        ds = LoadDataset('../data/train', MEAN, STD, length=TRAINNING_DS_SIZE)
        loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=CORES)
    

    for batch_images in tqdm(loader):
        batch_images = batch_images.to(DEVICE)
        d_loss = train_discriminator(G, D, downsampler, batch_images, LOSS_FN, d_optimizer, LATENT_SIZE, BATCH_SIZE, DEVICE)
        g_loss = train_generator(G, D, LOSS_FN, g_optimizer, LATENT_SIZE, BATCH_SIZE, DEVICE)
        
    d_loss, g_loss, r_score, f_score, fake_images = evaluate(G, D, downsampler, LOSS_FN, val_loader, LATENT_SIZE, BATCH_SIZE, DEVICE)
    d_losses.append(d_loss)
    g_losses.append(g_loss)
    r_scores.append(r_score)
    f_scores.append(f_score)
    
    torch.save(G.state_dict(), 'models/generator.pth')
    torch.save(D.state_dict(), 'models/discriminator.pth')
    
    d_loss = round(d_loss,5)
    g_loss = round(g_loss,5)
    r_score = round(r_score,5)
    f_score = round(f_score,5)

    d_optimizer.param_groups[0]['lr'] -= D_LR_DECAY
    g_optimizer.param_groups[0]['lr'] -= G_LR_DECAY

    elapsed = round(time.time()-start, 2)
    print(f"Epoch #{epochs+1}: d_loss={d_loss}, g_loss={g_loss}, r_score={r_score}, f_score={f_score} elapsed={elapsed} s")
    
    with open("log.json", "r") as fh:
        log = json.load(fh)
    log["epochs"] += 1
    log[epochs+1] = {"d_loss": d_loss, "g_loss": g_loss, "r_score": r_score, "f_score": f_score, "elapsed": elapsed}
    
    with open("log.json", "w") as fh:
        json.dump(log, fh)
    
    save_image(fake_images, f'output/{epochs+1}.png')

  0%|          | 0/5752 [00:00<?, ?it/s]

Epoch #1: d_loss=0.90493, g_loss=1.23562, r_score=0.61404, f_score=0.30757 elapsed=4917.68 s


  0%|          | 0/5752 [00:00<?, ?it/s]

Epoch #2: d_loss=0.52996, g_loss=2.26103, r_score=0.73851, f_score=0.16041 elapsed=4723.23 s


  0%|          | 0/5752 [00:00<?, ?it/s]

...

  0%|          | 0/5752 [00:00<?, ?it/s]

Epoch #36: d_loss=0.11973, g_loss=8.07066, r_score=0.89048, f_score=0.00299 elapsed=4721.48 s


  0%|          | 0/5752 [00:00<?, ?it/s]

Epoch #37: d_loss=0.10175, g_loss=13.4245, r_score=0.90355, f_score=1e-05 elapsed=4721.18 s


  0%|          | 0/5752 [00:00<?, ?it/s]